Need to do: 
1). Create Player History data (Last 3 years)
2). Create Get Live Current Season Data 
3). Create get methods for players 
4). Create sort methods by stats

In [12]:
pip install pandas requests 

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
### NOT IN USE AT THE MOMENT
###SCRAPING THE SEASON CUMULATIVE HISTORICAL DATA 

import requests
import pandas as pd
from io import StringIO

# Function to download and process player data for specific seasons
def download_and_process_player_data(player_name):
    history_url = f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/players/{player_name}/history.csv"
    
    # Try to download the history.csv file
    history_response = requests.get(history_url)
    
    if history_response.status_code == 200:
        # Load the CSV data into a DataFrame
        player_data = pd.read_csv(StringIO(history_response.text))
        
        # Filter data by seasons and save to different files
        for season in ['2020/21']:
            season_data = player_data[player_data['season_name'] == season]
            
            if not season_data.empty:
                # Add a new column for the player's name
                season_data.insert(0, 'Name', player_name)
                
                # Determine the file name based on the season
                file_name = f'player_data_{season.replace("/", "_")}.csv'
                
                # Append to the file for this season
                season_data.to_csv(file_name, mode='a', index=False, header=not pd.io.common.file_exists(file_name))
                print(f"Appended data for {player_name} to {file_name}.")
    else:
        print(f"Could not find history.csv for {player_name}")

# Function to get all player folders from the 2023-24 directory
def get_player_folders():
    url = "https://api.github.com/repos/vaastav/Fantasy-Premier-League/contents/data/2023-24/players"
    
    # Send a GET request to the GitHub API
    response = requests.get(url)

    if response.status_code == 200:
        player_folders = response.json()  # Get the list of player folders
        return [player['name'] for player in player_folders if player['type'] == 'dir']  # Return only folder names
    else:
        print("Failed to retrieve player folders.")
        return []

# Get all player folders for the 2023-24 season
player_folders = get_player_folders()

# Loop through each player folder and process data
for player in player_folders:
    download_and_process_player_data(player)

Appended data for Aaron_Connolly_127 to player_data_2020_21.csv.
Appended data for Aaron_Cresswell_530 to player_data_2020_21.csv.
Appended data for Aaron_Ramsdale_17 to player_data_2020_21.csv.
Appended data for Aaron_Wan-Bissaka_401 to player_data_2020_21.csv.
Appended data for Abdoulaye_Doucouré_249 to player_data_2020_21.csv.
Appended data for Adam_Lallana_138 to player_data_2020_21.csv.
Appended data for Adam_Webster_153 to player_data_2020_21.csv.
Appended data for Adama_Traoré_662 to player_data_2020_21.csv.
Appended data for Adrián_San Miguel del Castillo_289 to player_data_2020_21.csv.
Appended data for Aleksandar_Mitrović_278 to player_data_2020_21.csv.
Appended data for Alex_Iwobi_256 to player_data_2020_21.csv.
Appended data for Alex_Telles_370 to player_data_2020_21.csv.
Appended data for Alexis_Mac Allister_304 to player_data_2020_21.csv.
Appended data for Alisson_Ramses Becker_291 to player_data_2020_21.csv.
Appended data for Allan_Saint-Maximin_426 to player_data_2020_2

In [ ]:
import os
import requests

#SCRAPING THE FPL GW DATA 

def download_gw_csv_files(base_url, seasons, gw_range):
    """
    Downloads gameweek CSV files from the raw GitHub URL for specified seasons and gameweeks,
    and saves them in a directory in the current working directory.

    Parameters:
    base_url (str): The base URL for the raw GitHub content (e.g., 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/').
    seasons (list): A list of seasons (e.g., ['2021-22', '2022-23', '2023-24']).
    gw_range (range): The range of gameweeks to download (e.g., range(1, 39)).
    """
    # Get current working directory (CWD)
    cwd = os.getcwd()
    # Create a master folder named gw_data inside the CWD
    master_folder = os.path.join(cwd, 'gw_data')

    # Create master folder if it doesn't exist
    if not os.path.exists(master_folder):
        os.makedirs(master_folder)

    for season in seasons:
        # Create season folder inside master folder
        season_folder = os.path.join(master_folder, season)
        if not os.path.exists(season_folder):
            os.makedirs(season_folder)

        for gw in gw_range:
            gw_file = f'gw{gw}.csv'
            gw_url = f'{base_url}{season}/gws/{gw_file}'

            # Download the CSV file from the raw GitHub URL
            response = requests.get(gw_url)

            # Check if the request was successful
            if response.status_code == 200:
                # Save the file to the appropriate folder
                with open(os.path.join(season_folder, gw_file), 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded {gw_file} for {season}")
            else:
                print(f"Failed to download {gw_file} for {season}: {response.status_code} (URL: {gw_url})")

# Example usage
base_url = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/'
seasons = ['2021-22', '2022-23', '2023-24']
gw_range = range(1, 39)  # Gameweeks 1 to 38

download_gw_csv_files(base_url, seasons, gw_range)


In [ ]:
#SCRAPING THE UNDERSTAT GW DATA
import os
import requests
import pandas as pd

# Define the base URLs for players directory and CSV files
base_api_url = "https://api.github.com/repos/vaastav/Fantasy-Premier-League/contents/data/{}/players"
base_raw_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{}/players/{}/gw.csv"
seasons = ['2020-21', '2021-22', '2022-23', '2023-24']
base_directory = os.path.join(os.getcwd(), 'understat_gw_data')

# Create the base directory if it doesn't exist
if not os.path.exists(base_directory):
    os.makedirs(base_directory)

# Columns not to aggregate
columns_to_skip = [
    'fixture', 'kickoff_time', 'opponent_team', 'round',
    'team_a_score', 'team_h_score', 'transfers_balance',
    'transfers_in', 'transfers_out', 'value', 'was_home'
]

# Function to download and save CSV files
def download_and_save_csv(season, player_name):
    # Create the folder for the season
    season_folder = os.path.join(base_directory, season)
    if not os.path.exists(season_folder):
        os.makedirs(season_folder)
    
    # Format the player's name to save as First_Last.csv
    player_csv = f"{player_name.replace('_', ' ').title().replace(' ', '_')}.csv"
    output_path = os.path.join(season_folder, player_csv)

    # Construct the raw URL for the player's gws.csv
    player_url = base_raw_url.format(season, player_name)

    # Download the CSV
    try:
        response = requests.get(player_url)
        response.raise_for_status()
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {player_csv} for {season}")

        # Now process the CSV to add the aggregated row and game count
        process_csv(output_path)

    except requests.exceptions.RequestException as e:
        print(f"Failed to download {player_csv} for {season}: {e}")

# Function to process the CSV, aggregate stats, and count games played
def process_csv(file_path):
    try:
        # Read the CSV into a pandas DataFrame
        df = pd.read_csv(file_path)

        # Filter columns to aggregate (exclude columns in `columns_to_skip`)
        columns_to_aggregate = [col for col in df.columns if col not in columns_to_skip]

        # Add a new row with the sum of the relevant columns
        aggregate_row = df[columns_to_aggregate].sum(numeric_only=True)

        # Create an empty row for skipped columns and merge with the aggregate row
        empty_columns = {col: '' for col in columns_to_skip}
        combined_row = pd.concat([pd.Series(empty_columns), aggregate_row])

        # Add the combined row to the DataFrame
        df = pd.concat([df, pd.DataFrame([combined_row])], ignore_index=True)

        # Count the number of games where minutes > 0
        games_played = (df['minutes'] > 0).sum()

        # Add a note with the games played to the DataFrame as a new row
        games_played_row = pd.Series([f'Games played: {games_played}'] + [''] * (len(df.columns) - 1), index=df.columns)
        df = pd.concat([df, pd.DataFrame([games_played_row])], ignore_index=True)

        # Write the updated DataFrame back to CSV
        df.to_csv(file_path, index=False)
        print(f"Processed and saved: {file_path}")

    except Exception as e:
        print(f"Failed to process {file_path}: {e}")

# Function to get the list of players from the GitHub API
def get_players_list(season):
    season_url = base_api_url.format(season)
    try:
        response = requests.get(season_url)
        response.raise_for_status()
        player_folders = response.json()
        player_names = [folder['name'] for folder in player_folders if folder['type'] == 'dir']
        return player_names
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve player list for {season}: {e}")
        return []

# Loop over seasons and dynamically get players for each season
for season in seasons:
    print(f"Fetching players for season {season}...")
    player_names = get_players_list(season)
    
    if player_names:
        for player_name in player_names:
            download_and_save_csv(season, player_name)




In [13]:
### REORGANIZING THE SCRAPED FPL GW DATA into a player-based format 
import os
import pandas as pd

# Define the paths for the original and new directories
base_gw_directory = os.path.join(os.getcwd(), 'fpl_raw_data')
new_directory = os.path.join(os.getcwd(), 'fpl_gw_data')

# Create the new directory if it doesn't exist
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# Function to process each season and aggregate player data
def process_season(season):
    season_folder = os.path.join(base_gw_directory, season)
    new_season_folder = os.path.join(new_directory, season)
    
    # Create season subdirectory in the new directory if it doesn't exist
    if not os.path.exists(new_season_folder):
        os.makedirs(new_season_folder)
    
    # Dictionary to hold player data
    player_data = {}

    # Iterate over all gw{x}.csv files (gameweek files)
    for gw_file in sorted(os.listdir(season_folder)):
        if gw_file.endswith('.csv'):
            gw_path = os.path.join(season_folder, gw_file)
            df = pd.read_csv(gw_path)

            # Iterate through each row (player entry) in the gameweek file
            for index, row in df.iterrows():
                player_name = row['name']
                
                # If player_name is not in the dictionary, initialize an empty list for the player
                if player_name not in player_data:
                    player_data[player_name] = []

                # Append the row data (converted to a dictionary) to the player's list
                player_data[player_name].append(row)

    # Now save each player's data into their respective CSV files
    for player_name, rows in player_data.items():
        player_df = pd.DataFrame(rows)  # Convert the list of rows to a DataFrame
        player_csv_filename = f"{player_name.replace(' ', '_')}.csv"
        player_csv_path = os.path.join(new_season_folder, player_csv_filename)

        # Save the DataFrame to the player's CSV file
        player_df.to_csv(player_csv_path, index=False)

# Traverse all season folders in fpl_gw_data
for season in sorted(os.listdir(base_gw_directory)):
    season_path = os.path.join(base_gw_directory, season)
    if os.path.isdir(season_path):
        print(f"Processing season: {season}")
        process_season(season)

print("Player data has been successfully scraped and saved.")


Processing season: 2021-22
Processing season: 2022-23
Processing season: 2023-24
Player data has been successfully scraped and saved.
